In [152]:
import pandas as pd
import numpy as np
import requests
import config

In [153]:
#This is my google api key, it is the same key we used for gmaps but you have to add the geocoding api

g_key = config.g_key

In [154]:
#import that city bike data

bike_df = pd.read_csv("201908-citibike-tripdata.csv")
bike_df.head()

,Number To String,Number of Records (bin),Number of Records,Bikeid,Birth Year,End Station Id,End Station Latitude,End Station Longitude,End Station Name,Gender,Start Station Id,Start Station Latitude,Start Station Longitude,Start Station Name,Starttime,Stoptime,Tripduration,Usertype
0,UNKNOWN,1,1,25837,1959,305.0,40.760958,-73.967245,E 58 St & 3 Ave,0,229.0,40.727434,-73.993790,Great Jones St,8/8/2019 3:32:26 PM,8/8/2019 3:57:47 PM,1521,Customer
1,UNKNOWN,1,1,25837,1959,229.0,40.727434,-73.993790,Great Jones St,0,307.0,40.714275,-73.989900,Canal St & Rutgers St,8/8/2019 3:17:21 PM,8/8/2019 3:30:03 PM,761,Customer
2,UNKNOWN,1,1,31830,1959,2008.0,40.705693,-74.016777,Little West St & 1 Pl,0,405.0,40.739323,-74.008119,Washington St & Gansevoort St,8/8/2019 2:05:14 PM,8/8/2019 2:23:50 PM,1116,Customer
3,UNKNOWN,1,1,25837,1959,307.0,40.714275,-73.989900,Canal St & Rutgers St,0,427.0,40.701907,-74.013942,Bus Slip & State St,8/8/2019 2:56:59 PM,8/8/2019 3:15:06 PM,1087,Customer
4,UNKNOWN,1,1,19955,1959,525.0,40.755942,-74.002116,W 34 St & 11 Ave,0,514.0,40.760875,-74.002777,12 Ave & W 40 St,8/8/2019 12:38:41 PM,8/8/2019 12:44:21 PM,339,Customer


In [155]:
station_coords = bike_df.groupby("End Station Id").first()[["End Station Longitude", "End Station Latitude"]]
station_coords

,End Station Longitude,End Station Latitude
End Station Id,,
72.0,-73.993929,40.767272
79.0,-74.006667,40.719116
82.0,-74.000165,40.711174
83.0,-73.976323,40.683826
116.0,-74.001497,40.741776
...,...,...
3799.0,-73.988050,40.749640
3803.0,-73.956934,40.665816
3804.0,-73.986842,40.702461


In [156]:
station_coords

,End Station Longitude,End Station Latitude
End Station Id,,
72.0,-73.993929,40.767272
79.0,-74.006667,40.719116
82.0,-74.000165,40.711174
83.0,-73.976323,40.683826
116.0,-74.001497,40.741776
...,...,...
3799.0,-73.988050,40.749640
3803.0,-73.956934,40.665816
3804.0,-73.986842,40.702461


In [157]:
def getZipCodeFromLatLng(lat, lng):
    
    api_endpoint = "https://maps.googleapis.com/maps/api/geocode/json?"

    url_query = f"{api_endpoint}latlng={lat},{lng}&key={g_key}"
    
    try:
        response = requests.get(url_query)
        
    except Exception as e:
        print("Network Error" + e.message)
        return -1
    
    if( response.status_code is 200 ):
        
        response_json = response.json()
        
        #Yeah it is convoluted but the JSON is pretty complicated
        #as the number of address componenets changes - we just find
        #which one has the type postal_code
        address = response_json["results"][0]["address_components"]
        postal_code_info = [x for x in address if x["types"] == ["postal_code"]]
        zip_code = postal_code_info[0]["long_name"]
        
        return zip_code
    
    else:
        return -1



In [158]:
import time

def getZipCodesUsingGoogleGeocodeApi(coordinates):
    
    #it will time out if we make the requests too fast
    counter = 0
    zip_codes = []
    
    for coordinate in coordinates:
        
        lat = coordinate[0]
        lng = coordinate[1]
        
        zip_codes.append(getZipCodeFromLatLng(lat,lng))
        
        #Google APIs rate limit so doing 30 at a time
        if counter % 30 == 0:
            time.sleep(2)
            print(counter)
        
        counter += 1
        
    return zip_codes

In [159]:

lats = station_coords["End Station Latitude"].tolist()
lngs = station_coords["End Station Longitude"].tolist()

lat_lngs = list(zip(lats[:], lngs[:]))

zip_codes = getZipCodesUsingGoogleGeocodeApi(lat_lngs)

0
30
60
90
120
150
180
210
240
270
300
330
360
390
420
450
480
510
540
570
600
630
660
690
720
750
780


In [161]:
#add the zip codes

station_coords["zip_codes"] = zip_codes
station_coords = station_coords.reset_index()

#merge back into the citybike data frame
#we don't want to include the lat and long again so only select two columns

#bike_df = bike_df.merge(station_coords[["End Station Id", "zip_codes"]], on="End Station Id")
#bike_df.tail()

In [ ]:
#PART 2:  GETTING DEMOGRAPHIC DATA FOR THE ZIP CODES IN THIS DATA SET.  
#SAVING THIS TO CSV SO STUDENTS CAN PRACTICE MERGING

In [64]:
datafinder_key = config.datafinder_key

#NOTE:  THIS NEEDS REFACTORING, NORMALLY WOULD BREAK INTO TESTABLE FUNCTIONS
#FOLLOWING THE SINGLE RESPONSIBILITY PRINCIPLE

def getDemographicsForZipUsingDatafinder(zip_code):
                 
    datafinder_endpoint = "https://api.datafinder.com/v2/qdf.php?"
    datafinder_query = f"{datafinder_endpoint}k2={datafinder_key}&service=zip5&d_zip={zip_code}"

    response = requests.get(datafinder_query)
    
    try:
        if response.status_code is 200:
            return response.json()
        else:
            return {}
    except Exception as e:
        print(e)
        print("Oh No API Error")
        return {}
    

In [138]:
def extractFieldsFromDemographics(demographics_arr):

    total_pop = []
    pop_density = []
    percent_bachelors_holders = []
    percent_in_poverty = []
    percent_females = []
    percent_males = []
    
    
    for zip_demographics in demographics_arr:
        
        #if there are no results I want to add nan rather than 0
        total = np.nan
        density = np.nan
        percent_bach_holders =  np.nan
        percent_poverty = np.nan
        percent_female = np.nan
        percent_male = np.nan
        
        
        if("datafinder" not in zip_demographics or "results" not in zip_demographics["datafinder"]):
            print("No results")
        else:
            try:
                data = zip_demographics["datafinder"]["results"][0]
                total = int(data["Population112018"])
                density = float(data["PopulationDensity"])
                total_bach_holders = int(data["EducationBachelorsDegreeFemalePop25"])\
                                    +int(data["EducationBachelorsDegreeMalePop25"])
                percent_bach_holders = total_bach_holders /  total * 100
                percent_poverty = int(data["PopulationinPovertyTotal"]) / total * 100
                total_male = int(data["PopulationMaleNotinPovertyTotal"])\
                            +int(data["PopulationMaleinPovertyTotal"])
                percent_male = total_male / total * 100
                total_female = int(data["PopulationFemaleinPovertyTotal"])\
                              +int(data["PopulationFemaleNotinPovertyTotal"])
                percent_female = total_female / total * 100
                
                print(percent_male, percent_female)
            except:
                pass
        total_pop.append(total)
        pop_density.append(density)
        percent_bachelors_holders.append(percent_bach_holders)
        percent_in_poverty.append(percent_poverty)
        percent_females.append(percent_female)
        percent_males.append(percent_male)
    
    return total_pop, pop_density, percent_bachelors_holders, percent_in_poverty, percent_females, percent_males

In [56]:
#The fields it returns

{'datafinder': {'version': '2.0',
  'query-id': 'f5fe245965c5d2f62e309b220abbc280',
  'results': [{'AreaCode': '310',
    'AsianAloneHhlds1': '116',
    'AsianAloneHhlds2': '154',
    'AsianAloneHhlds3': '93',
    'AsianAloneHhlds4': '89',
    'AsianAloneHhlds5': '29',
    'AsianAloneHhlds6': '9',
    'AsianAloneHhlds7orMore': '5',
    'AsianNonHispanicHeadofHhlds': '424',
    'BetweenGulfWarandVietnamEraonly': '2',
    'BetweenVietnamEraandKoreanWaronly': '244',
    'BlackAloneHhlds1': '74',
    'BlackAloneHhlds2': '52',
    'BlackAloneHhlds3': '20',
    'BlackAloneHhlds4': '15',
    'BlackAloneHhlds5': '6',
    'BlackAloneHhlds6': '2',
    'BlackAloneHhlds7orMore': '3',
    'BlackNonHispanicHeadofHhlds': '144',
    'CBSACode': '31080',
    'CBSAType1Metropolitan2Micropolitan': '1',
    'CBSAVerbose': 'Los Angeles-Long Beach-Anaheim, CA',
    'CityCode': '606308',
    'CityType': 'C1',
    'City': 'Beverly Hills city,',
    'CountyFIPSCode': '6037',
    'County': 'Los Angeles, CA',
  

In [145]:

def addDemographicDataForZips(zip_codes):
    
    print(zip_codes)
    demographic_data = []
    
    counter = 0
    
    for zip_code in zip_codes[:]:
        
        data = getDemographicsForZipUsingDatafinder(zip_code)
        print(data)
        
        demographic_data.append(data)
        
        counter += 1
        
        if counter % 30 is 0:
            time.sleep(2)
            print("sleeping " + str(counter))
    
    #extract fields
    total_pop, pop_density, percent_bachelors_holders,\
    percent_in_poverty, percent_females, percent_males = extractFieldsFromDemographics(demographic_data)
    
    #create df
    zip_demographics = pd.DataFrame({
        "zip_code": zip_codes,
        "Total Population": total_pop,
        "Percent Female": percent_females,
        "Percent Male": percent_males,
        "Percent Bachelors Holders": percent_bachelors_holders,
        "Percent In Poverty": percent_in_poverty
    })
    
    return zip_demographics
    

In [172]:
unique_zips = set(zip_codes)
len(unique_zips)

63

In [146]:
zipcode_demographics_df = addDemographicDataForZips(unique_zips)

['10022' '10012' '10004' '10002' '10001' '10026' '10018' '10027' '10023'
 '10028' '10282' '10011' '10036' '10019' '10280' '10021' '10069' '10024'
 '10010' '10007' '10013' '11249' '11232' '11201' '11216' '11217' '11215'
 '11238' '11237' '10128' '11205' '10014' '10003' '10009' '11211' '11225'
 '10017' '10065' '10075' '10016' '10168' '10025' '10038' '11101' '11231'
 '10029' '11206' '10175' '10005' '10035' '11222' '11106' '11102' '11221'
 '11103' '11233' '11105' '11213' '07310' '07302' '07307' '07304' '07306']
['10022' '10012' '10004' '10002' '10001' '10026' '10018' '10027' '10023'
 '10028' '10282' '10011' '10036' '10019' '10280' '10021' '10069' '10024'
 '10010' '10007' '10013' '11249' '11232' '11201' '11216' '11217' '11215'
 '11238' '11237' '10128' '11205' '10014' '10003' '10009' '11211' '11225'
 '10017' '10065' '10075' '10016' '10168' '10025' '10038' '11101' '11231'
 '10029' '11206' '10175' '10005' '10035' '11222' '11106' '11102' '11221'
 '11103' '11233' '11105' '11213' '07310' '07302' '

{'datafinder': {'version': '2.0', 'query-id': 'a45101172acfd8b4af811036e07316eb', 'results': [{'AreaCode': '212', 'AsianAloneHhlds1': '831', 'AsianAloneHhlds2': '637', 'AsianAloneHhlds3': '233', 'AsianAloneHhlds4': '142', 'AsianAloneHhlds5': '51', 'AsianAloneHhlds6': '17', 'AsianAloneHhlds7orMore': '17', 'AsianNonHispanicHeadofHhlds': '1924', 'BetweenGulfWarandVietnamEraonly': '26', 'BetweenVietnamEraandKoreanWaronly': '61', 'BlackAloneHhlds1': '166', 'BlackAloneHhlds2': '64', 'BlackAloneHhlds3': '21', 'BlackAloneHhlds4': '22', 'BlackAloneHhlds5': '1', 'BlackAloneHhlds7orMore': '2', 'BlackNonHispanicHeadofHhlds': '246', 'CBSACode': '35620', 'CBSAType1Metropolitan2Micropolitan': '1', 'CBSAVerbose': 'New York-Newark-Jersey City, NY-NJ-PA', 'CityCode': '3651000', 'CityType': 'C1', 'City': 'New York city,', 'CountyFIPSCode': '36061', 'County': 'New York, NY', 'EducationAssociatesDegreeFemalePop25': '144', 'EducationAssociatesDegreeMalePop25': '229', 'EducationBachelorsDegreeFemalePop25': '

{'datafinder': {'version': '2.0', 'query-id': 'fca0d8c930fd4a88dc3dcc90c9aace63', 'results': [{'AreaCode': '212', 'AsianAloneHhlds1': '88', 'AsianAloneHhlds2': '79', 'AsianAloneHhlds3': '32', 'AsianAloneHhlds4': '16', 'AsianAloneHhlds5': '4', 'AsianAloneHhlds6': '1', 'AsianNonHispanicHeadofHhlds': '214', 'BetweenGulfWarandVietnamEraonly': '20', 'BlackAloneHhlds1': '21', 'BlackAloneHhlds2': '11', 'BlackAloneHhlds3': '5', 'BlackAloneHhlds4': '2', 'BlackAloneHhlds5': '1', 'BlackNonHispanicHeadofHhlds': '34', 'CBSACode': '35620', 'CBSAType1Metropolitan2Micropolitan': '1', 'CBSAVerbose': 'New York-Newark-Jersey City, NY-NJ-PA', 'CityCode': '3651000', 'CityType': 'C1', 'City': 'New York city,', 'CountyFIPSCode': '36061', 'County': 'New York, NY', 'EducationAssociatesDegreeFemalePop25': '44', 'EducationAssociatesDegreeMalePop25': '70', 'EducationBachelorsDegreeFemalePop25': '377', 'EducationBachelorsDegreeMalePop25': '270', 'EducationBachelorsDegreeorHigherPop25AsianFemale': '151', 'Education

{'datafinder': {'version': '2.0', 'query-id': 'edf9767e1cc72b4afaac4e03b2ca1d2c', 'results': [{'AreaCode': '212', 'AsianAloneHhlds1': '3213', 'AsianAloneHhlds2': '3715', 'AsianAloneHhlds3': '2166', 'AsianAloneHhlds4': '1923', 'AsianAloneHhlds5': '1190', 'AsianAloneHhlds6': '516', 'AsianAloneHhlds7orMore': '532', 'AsianNonHispanicHeadofHhlds': '12973', 'BetweenGulfWarandVietnamEraonly': '206', 'BetweenKoreanWarandWorldWarIIonly': '5', 'BetweenVietnamEraandKoreanWaronly': '166', 'BlackAloneHhlds1': '1121', 'BlackAloneHhlds2': '814', 'BlackAloneHhlds3': '419', 'BlackAloneHhlds4': '330', 'BlackAloneHhlds5': '134', 'BlackAloneHhlds6': '66', 'BlackAloneHhlds7orMore': '47', 'BlackNonHispanicHeadofHhlds': '2279', 'CBSACode': '35620', 'CBSAType1Metropolitan2Micropolitan': '1', 'CBSAVerbose': 'New York-Newark-Jersey City, NY-NJ-PA', 'CityCode': '3651000', 'CityType': 'C1', 'City': 'New York city,', 'CountyFIPSCode': '36061', 'County': 'New York, NY', 'EducationAssociatesDegreeFemalePop25': '1242

{'datafinder': {'version': '2.0', 'query-id': '9d03ef02738bd9e4a3e03052d2bcf268', 'results': [{'AreaCode': '212', 'AsianAloneHhlds1': '931', 'AsianAloneHhlds2': '638', 'AsianAloneHhlds3': '206', 'AsianAloneHhlds4': '89', 'AsianAloneHhlds5': '26', 'AsianAloneHhlds6': '7', 'AsianAloneHhlds7orMore': '1', 'AsianNonHispanicHeadofHhlds': '1940', 'BetweenGulfWarandVietnamEraonly': '79', 'BetweenKoreanWarandWorldWarIIonly': '13', 'BetweenVietnamEraandKoreanWaronly': '63', 'BlackAloneHhlds1': '456', 'BlackAloneHhlds2': '196', 'BlackAloneHhlds3': '87', 'BlackAloneHhlds4': '50', 'BlackAloneHhlds5': '13', 'BlackAloneHhlds6': '16', 'BlackAloneHhlds7orMore': '10', 'BlackNonHispanicHeadofHhlds': '729', 'CBSACode': '35620', 'CBSAType1Metropolitan2Micropolitan': '1', 'CBSAVerbose': 'New York-Newark-Jersey City, NY-NJ-PA', 'CityCode': '3651000', 'CityType': 'C1', 'City': 'New York city,', 'CountyFIPSCode': '36061', 'County': 'New York, NY', 'EducationAssociatesDegreeFemalePop25': '358', 'EducationAssoci

...

{'datafinder': {'version': '2.0', 'query-id': 'bd6572770d5283eef5e4f9285868bc97', 'results': [{'AreaCode': '718', 'AsianAloneHhlds1': '72', 'AsianAloneHhlds2': '48', 'AsianAloneHhlds3': '27', 'AsianAloneHhlds4': '45', 'AsianAloneHhlds5': '16', 'AsianAloneHhlds6': '7', 'AsianAloneHhlds7orMore': '12', 'AsianNonHispanicHeadofHhlds': '213', 'BetweenGulfWarandVietnamEraonly': '213', 'BetweenKoreanWarandWorldWarIIonly': '2', 'BetweenVietnamEraandKoreanWaronly': '126', 'BlackAloneHhlds1': '6060', 'BlackAloneHhlds2': '4769', 'BlackAloneHhlds3': '3418', 'BlackAloneHhlds4': '2044', 'BlackAloneHhlds5': '1193', 'BlackAloneHhlds6': '467', 'BlackAloneHhlds7orMore': '458', 'BlackNonHispanicHeadofHhlds': '17404', 'CBSACode': '35620', 'CBSAType1Metropolitan2Micropolitan': '1', 'CBSAVerbose': 'New York-Newark-Jersey City, NY-NJ-PA', 'CityCode': '3651000', 'CityType': 'C1', 'City': 'New York city,', 'CountyFIPSCode': '36047', 'County': 'Kings, NY', 'EducationAssociatesDegreeFemalePop25': '1483', 'Educati

{'datafinder': {'version': '2.0', 'query-id': '1e233dc3c548715c495968e844be8818', 'results': [{'AreaCode': '201', 'AsianAloneHhlds1': '1413', 'AsianAloneHhlds2': '1935', 'AsianAloneHhlds3': '880', 'AsianAloneHhlds4': '416', 'AsianAloneHhlds5': '105', 'AsianAloneHhlds6': '40', 'AsianAloneHhlds7orMore': '21', 'AsianNonHispanicHeadofHhlds': '4846', 'BetweenGulfWarandVietnamEraonly': '187', 'BetweenKoreanWarandWorldWarIIonly': '2', 'BetweenVietnamEraandKoreanWaronly': '27', 'BlackAloneHhlds1': '772', 'BlackAloneHhlds2': '501', 'BlackAloneHhlds3': '288', 'BlackAloneHhlds4': '168', 'BlackAloneHhlds5': '61', 'BlackAloneHhlds6': '29', 'BlackAloneHhlds7orMore': '32', 'BlackNonHispanicHeadofHhlds': '1815', 'CBSACode': '35620', 'CBSAType1Metropolitan2Micropolitan': '1', 'CBSAVerbose': 'New York-Newark-Jersey City, NY-NJ-PA', 'CityCode': '3436000', 'CityType': 'C5', 'City': 'Jersey City city,', 'CountyFIPSCode': '34017', 'County': 'Hudson, NJ', 'EducationAssociatesDegreeFemalePop25': '416', 'Educa

sleeping 60


{'datafinder': {'version': '2.0', 'query-id': '32d05d5ed8589feeefb77dc2131bf061', 'results': [{'AreaCode': '201', 'AsianAloneHhlds1': '338', 'AsianAloneHhlds2': '641', 'AsianAloneHhlds3': '676', 'AsianAloneHhlds4': '705', 'AsianAloneHhlds5': '247', 'AsianAloneHhlds6': '151', 'AsianAloneHhlds7orMore': '109', 'AsianNonHispanicHeadofHhlds': '2889', 'BetweenGulfWarandVietnamEraonly': '119', 'BetweenKoreanWarandWorldWarIIonly': '12', 'BetweenVietnamEraandKoreanWaronly': '133', 'BlackAloneHhlds1': '436', 'BlackAloneHhlds2': '318', 'BlackAloneHhlds3': '259', 'BlackAloneHhlds4': '213', 'BlackAloneHhlds5': '72', 'BlackAloneHhlds6': '27', 'BlackAloneHhlds7orMore': '33', 'BlackNonHispanicHeadofHhlds': '1110', 'CBSACode': '35620', 'CBSAType1Metropolitan2Micropolitan': '1', 'CBSAVerbose': 'New York-Newark-Jersey City, NY-NJ-PA', 'CityCode': '3436000', 'CityType': 'C5', 'City': 'Jersey City city,', 'CountyFIPSCode': '34017', 'County': 'Hudson, NJ', 'EducationAssociatesDegreeFemalePop25': '844', 'Edu

{'datafinder': {'version': '2.0', 'query-id': 'f2457887156ff0fabeed31c277524456', 'results': [{'AreaCode': '201', 'AsianAloneHhlds1': '1154', 'AsianAloneHhlds2': '1678', 'AsianAloneHhlds3': '1338', 'AsianAloneHhlds4': '1094', 'AsianAloneHhlds5': '471', 'AsianAloneHhlds6': '225', 'AsianAloneHhlds7orMore': '222', 'AsianNonHispanicHeadofHhlds': '6079', 'BetweenGulfWarandVietnamEraonly': '288', 'BetweenKoreanWarandWorldWarIIonly': '14', 'BetweenVietnamEraandKoreanWaronly': '128', 'BlackAloneHhlds1': '1167', 'BlackAloneHhlds2': '697', 'BlackAloneHhlds3': '462', 'BlackAloneHhlds4': '355', 'BlackAloneHhlds5': '160', 'BlackAloneHhlds6': '59', 'BlackAloneHhlds7orMore': '55', 'BlackNonHispanicHeadofHhlds': '2789', 'CBSACode': '35620', 'CBSAType1Metropolitan2Micropolitan': '1', 'CBSAVerbose': 'New York-Newark-Jersey City, NY-NJ-PA', 'CityCode': '3436000', 'CityType': 'C5', 'City': 'Jersey City city,', 'CountyFIPSCode': '34017', 'County': 'Hudson, NJ', 'EducationAssociatesDegreeFemalePop25': '963'

In [147]:
zipcode_demographics_df.head()

,zip_codes,Total Population,Percent Female,Percent Male,Percent Bachelors Holders,Percent In Poverty
0,10022,34776.0,52.884173,43.998735,33.390844,4.005636
1,10012,26151.0,49.130052,46.755382,31.727276,10.557914
2,10004,2237.0,48.100134,49.128297,28.922664,6.616004
3,10002,87686.0,51.324043,45.575120,14.969322,27.284857
4,10001,23530.0,49.694008,45.414365,24.713132,13.697408


In [151]:
station_coords

,End Station Longitude,End Station Latitude
End Station Id,,
72.0,-73.993929,40.767272
79.0,-74.006667,40.719116
82.0,-74.000165,40.711174
83.0,-73.976323,40.683826
116.0,-74.001497,40.741776
...,...,...
3799.0,-73.988050,40.749640
3803.0,-73.956934,40.665816
3804.0,-73.986842,40.702461


In [162]:
station_zip_demographics_df = station_coords.merge(df, on="zip_codes")
station_zip_demographics_df

,End Station Id,End Station Longitude,End Station Latitude,zip_codes,Total Population,Percent Female,Percent Male,Percent Bachelors Holders,Percent In Poverty
0,72.0,-73.993929,40.767272,10019,43687.0,48.481242,48.646050,29.180305,11.495411
1,173.0,-73.984527,40.760683,10019,43687.0,48.481242,48.646050,29.180305,11.495411
2,281.0,-73.973715,40.764397,10019,43687.0,48.481242,48.646050,29.180305,11.495411
3,422.0,-73.988038,40.770513,10019,43687.0,48.481242,48.646050,29.180305,11.495411
4,423.0,-73.986905,40.765849,10019,43687.0,48.481242,48.646050,29.180305,11.495411
...,...,...,...,...,...,...,...,...,...
802,3775.0,-73.923769,40.702013,11237,50939.0,46.192505,50.320972,6.753175,28.237696
803,3777.0,-73.923044,40.699304,11237,50939.0,46.192505,50.320972,6.753175,28.237696
804,3778.0,-73.920950,40.702700,11237,50939.0,46.192505,50.320972,6.753175,28.237696
805,3779.0,-73.919911,40.704876,11237,50939.0,46.192505,50.320972,6.753175,28.237696


In [165]:
station_zip_demographics_df.to_csv("station_zipcode_demographics.csv")